# Parallel Computing

In this lesson we will deal with parallel computing, which is a type of computation in which many calculations or the execution of processes are carried out simultaneously on different CPU cores. We will show the differences between multi-threading and multi-processing and we will learn how those techniques are implemented in Julia.

For this lesson you will need Julia version 1.3 or above.

## Contents

- [Intro](#Intro)
- [Data based](#Data-based)
- [Task based](#Task-based)


## Intro 
In order to use multi-threading we need to start Julia with a number of threads equal to the number of you CPU cores. If you are using the Juno IDE it will automatically start Julia with the appropriate number of threads. If you are working from the REPL, you need to manually start Julia from the command line.

```
$ export JULIA_NUM_THREADS=4
$ julia
```

In [ ]:
# Check number of threads
Threads.nthreads()

## Data based

The first step to know if it's worth applying parallel processing is to measure any potential benefit by timing bottlenecks in code. 
It is possible to measure the execution time of a function using the macro `@time` from the `BenchmarkTools` package.

In [ ]:
using SpecialFunctions
using BenchmarkTools

function besselFun()
    x = range(0,1000, length=10000000)
    results = zeros(length(x))
    results .= besselj1.(x)
    return
end

@time besselFun()

Rewrite this funciton in a loop

In [ ]:
function besselFun()
    x = range(0,1000, length=10000000)
    results = zeros(length(x))
    for i in 1:length(x)
       results[i] = besselj1(x[i])
    end
    return
end

@time besselFun()

In that loop every iteration is independent from the next one: this hints the possibility to make the code parallel. To achieve parallelization, we import the Threads module and call the `@threads` macro.

In [ ]:
function besselFun()
    x = range(0,1000, length=10000000)
    results = zeros(length(x))
Threads.@threads for i in 1:length(x)
            results[i] = besselj1(x[i])
         end
    return
end

@time besselFun()

## Task based

Use the `@spawn` macro and the fetch function


In [ ]:
import Base.Threads.@spawn
using BenchmarkTools

function slow_func(x)
    sleep(0.01) #sleep for 5ms
    return x
end

println("Time for @spawn/fetch code:")
@time let
    a = @spawn slow_func(2)
    b = @spawn slow_func(4)
    c = @spawn slow_func(42)
    d = @spawn slow_func(12)
    res = fetch(a) .+ fetch(b) .* fetch(c) ./ fetch(d)
    println("Result: ", res)
end

println("Time for serial code:")
@time let
    a = slow_func(2)
    b = slow_func(4)
    c = slow_func(42)
    d = slow_func(12)
    res = a .+ b .* c ./ d
    println("Result: ", res)
end